# Test

In [4]:
import adodbapi



partial_date = '11sept'

conn_str = (
    r'Provider=Microsoft.SQLSERVER.CE.OLEDB.4.0; Data Source=DataStore.sdf;'
)
connection = adodbapi.connect(conn_str)

cursor = connection.cursor()

search_term = "%" + partial_date + "%"
query_job = f"SELECT id FROM Job WHERE Name LIKE '{search_term}'"
cursor.execute(query_job)

results = cursor.fetchall()

for row in results:
    print(row)




('39',)
('40',)
('41',)
('42',)


In [5]:
search_term = 'test'

conn_str = r'Provider=Microsoft.SQLSERVER.CE.OLEDB.4.0; Data Source=DataStore.sdf;'
connection = adodbapi.connect(conn_str)
cursor = connection.cursor()

# Fetch job_id based on the search term

search_term = "%" + partial_date + "%"
query_job = f"SELECT id FROM Job WHERE Name LIKE '{search_term}'"
cursor.execute(query_job)
job_ids = [row[0] for row in cursor.fetchall()]
print(job_ids)


[39, 40, 41, 42]


# Data base 

In [6]:
import adodbapi

def search_database(search_term, well_name):
    conn_str = r'Provider=Microsoft.SQLSERVER.CE.OLEDB.4.0; Data Source=DataStore.sdf;'
    connection = adodbapi.connect(conn_str)
    cursor = connection.cursor()

    # Fetch job_id based on the search term
    search_term = "%" + search_term + "%"
    query_job = f"SELECT id FROM Job WHERE Name LIKE '{search_term}'"
    cursor.execute(query_job)
    job_ids = [row[0] for row in cursor.fetchall()]

    paths = []

    for job_id in job_ids:
        scan_query = f"SELECT OrderIndex FROM ScanArea WHERE AcquireSettings_id = {job_id} AND Name = '{well_name}'"
        cursor.execute(scan_query)
        
        for orderindex_row in cursor.fetchall():
            orderindex = orderindex_row[0]

            scan_query = f"SELECT Enabled FROM ScanArea WHERE AcquireSettings_id = {job_id} AND Name = '{well_name}'"
            cursor.execute(scan_query)
            for row in cursor.fetchall():
                if row[0] == 1:
                    full_path = f"{job_id}/Acquire_0/{orderindex}"
                    paths.append(full_path)

    cursor.close()
    connection.close()

    return paths


In [4]:
search_database('sept','B2')

['39/Acquire_0/22', '40/Acquire_0/22', '41/Acquire_0/22', '42/Acquire_0/22']

In [7]:
import tkinter as tk
from tkinter import ttk

def on_search():
    search_term = search_term_entry.get()
    well_name = well_name_entry.get()
    results = search_database(search_term, well_name)
    
    # Clearing the results box
    results_listbox.delete(0, tk.END)
    
    for result in results:
        results_listbox.insert(tk.END, result)

# GUI setup
app = tk.Tk()
app.title("Search Database")

# Search term
search_term_label = ttk.Label(app, text="Search Term:")
search_term_label.grid(column=0, row=0, padx=10, pady=10, sticky=tk.W)

search_term_entry = ttk.Entry(app)
search_term_entry.grid(column=1, row=0, padx=10, pady=10, sticky=tk.W + tk.E)

# Well name
well_name_label = ttk.Label(app, text="Well Name:")
well_name_label.grid(column=0, row=1, padx=10, pady=10, sticky=tk.W)

well_name_entry = ttk.Entry(app)
well_name_entry.grid(column=1, row=1, padx=10, pady=10, sticky=tk.W + tk.E)

# Search button
search_button = ttk.Button(app, text="Search", command=on_search)
search_button.grid(column=1, row=2, padx=10, pady=10)

# Results listbox
results_listbox = tk.Listbox(app, width=50, height=10)
results_listbox.grid(column=0, row=3, columnspan=2, padx=10, pady=10)

# Run the GUI loop
app.mainloop()
